# Pipeline PSF subtraction of MIRI coronagraphic data

Author: Jonathan Aguilar (jaguilar@stsci.edu)
Date: July 31, 2023

Requirements not part of the Python Standard Library:
- numpy https://numpy.org/
- matplotlib https://matplotlib.org/
- pandas https://pandas.pydata.org/
- astropy https://docs.astropy.org/en/stable/
- astroquery https://astroquery.readthedocs.io/en/latest/
- jwst https://jwst-pipeline.readthedocs.io/en/latest/

This two-purpose tutorial will show you how to retrieve data from MAST, and how to run the calibration pipeline's Stage 3 coronagraphy step on MIRI data.

In [ ]:
import os, sys
import requests
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt

from astroquery.mast import Mast
from astropy.time import Time

from astropy.io import fits

## Step 1: Retrieve MIRI/CORON data for HIP-65426 from program JWST-ERS-01386

The first step in this tutorial is to download the relevant data from MAST, using the MAST API. In the interest of brevity, I have left the code in a state in which it can be simply run, with only a minimal explanation of how it works or how to change parameters. 

More information on how to use the astroquery interface to MAST can be found here: 
- The MAST API: https://mast.stsci.edu/api/v0/
- The astroquery MAST interface: https://astroquery.readthedocs.io/en/latest/mast/mast.html

We're going to download our data strictly using the program ID and observation identifiers, taken from APT.

APT files are public, so you can open APT and do `File -> Retrieve from STScI -> Retrieve using Program ID`. Type "1386" into the entry field and press `Enter`. 

In [ ]:
prog_id = '1386'
observations = [4, 5, 6, 7, 8, 9, 28, 29, 30, 31]
observations = [str(i) for i in observations]

In [ ]:
def set_params(parameters):
    return [{"paramName":p, "values":v} for p,v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

In [ ]:
# put the search keyword parameters into a format that can query MAST
keywords = {
        'program': [prog_id], 
        'observtn': observations,
        'date_obs_mjd': [set_mjd_range('2021-03-01T00:00:00','2023-07-01T00:00:00')]
    }
params = {
    'columns': '*',
    'filters': set_params(keywords)
}

In this step, we request the metadata of the all data products related to our query. We will then select which subset of the products we want, and retrieve only those.    

In [ ]:
service = 'Mast.Jwst.Filtered.Miri'
t = Mast.service_request(service, params).to_pandas()
print(len(t))

If `t` has nothing in it (i.e. `len(t) == 0`), your query didn't work and you need to revise your keywords.

For this example, we only want the Stage 2b, flux-calibrated files. Pull out the names of the products we want to download. 

Furthermore, for Stage 3 processing, we want to download the ASN pool files and the ASN Table files

In [ ]:
# Preview the first 5 lines of the dataframe
t.head()

There are a lot of columns available to help you choose which data to retrieve! We want:
- cal/calints data: `productLevel == '2b'`
    - These are stored under the column `filename`
- Association files for Stage 3: `productLevel == '3'`
    - These are stored under the columns `asntable` and `asnpool`


In [ ]:
# cal files
products = list(t.query("productLevel == '2b'")['filename']) 
# asn files
products = products + list([i for i in t.query("productLevel == '3'")['asntable'] if 'coron3' in i])
# pool files
products = products + list(t.query("productLevel == '3'")['asnpool'])
print(f"We are going to retrieve the following {len(products)} files:\n")
for p in sorted(products):
    print("\t- "+p)

The following cell displays the constructed parameter object to illustrate the syntax for the query, which is described formally [here](https://mast.stsci.edu/api/v0/_services.html#MastScienceInstrumentKeywordsNircam). 

The full selection of keywords upon which to build search criteria is described in the [Field Descriptions for JWST Instrument Keywords](https://mast.stsci.edu/api/v0/_jwst_inst_keywd.html). Note that [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) parameter names do not always match the FITS keyword names. 

## SI Keyword Search
<a id="KW Search"></a>

This type of query is a little more primitive in [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) than that for the `Observation` class. Begin by specifying the web service for the query, which for this case is the [SI keyword search for Miri](https://mast.stsci.edu/api/v0/_services.html#MastScienceInstrumentKeywordsMiri). Then execute the query with arguments for the service and the search parameters.

In [ ]:
mast_url='https://mast.stsci.edu/api/v0.1/Download/file'
uri_prefix = 'mast:JWST/product/'
# tokens are only needed for proprietary data. Program 1386 is publicly available. Uncomment the line below if necessary.
# token = 'paste-token-here'

Output into a storage directory, `data-bkp`. We will then copy the files into the directory containing this notebook, so that they can be read by the association file downloaded from MAST. 

With the original files kept in a separate folder, we can easily restore the working files to their original state if they get modified. 

In [ ]:
out_dir = "./data-bkp"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
elif not os.access(out_dir, os.W_OK):
    raise ValueError(f"Output directory {out_dir} is not writable")

Finally, download the data products. There is a condition to skip downloading a file if it already exists. If you want to force a download, either delete the existing files or remove the condition.

In [ ]:
block_dataze = 1024000
for i, p in enumerate(products):
    r = requests.get(mast_url, params=dict(uri=uri_prefix+p), stream=True
                # include the following argument if authentication is needed
                #, headers=dict(Authorization=f"token {token}")
                    )
    # check for mal-formed HTTP requests
    try:
        r.raise_for_status()
    except Exception as e:
        print(e)
        print(f"Download failed for {p}")
        continue
    outfile = os.path.join(out_dir, p)

    if os.path.isfile(outfile):
        # file already exists, skip downloading it.
        print(f"{outfile} already exists, skipping.")
        continue
    else:
        with open(outfile, 'wb') as fd:
            for data in r.iter_content(chunk_size=block_size):
                fd.write(data)
    
    if not os.path.isfile(outfile):
        print("ERROR: " + outfile + " failed to download.")
    else:
        print(f"COMPLETE: ", outfile)

The data have been placed in a subfolder of this directory, called `data-bkp`. For `calwebb_coron3`, the data need to be in the same folder as the notebook. 

I suggest you copy the data to the working folder, so that the original versions can be restored if the images are modified (intentionally or accidentally), without having to download them again from MAST.

In [ ]:
# this works on Unix-like systems like macOS and Linux. Not sure about Windows.
!cp ./data-bkp/* .

## Part 2: PSF subtraction with `calwebb_coron3`

This divides the PSF subtraction process up into the following steps:
- data organizing
- data inspection
- background subtraction (if necessary)
- running pipeline Stage 3

### Split up the files into the different components

We're going to read all the `PRI` headers and put them all into a Pandas DataFrame, which will allow us to make use of DataFrame filtering tools.

In [ ]:
data_files = sorted(Path(out_dir).glob("*fits"))
len(data_files)

In [ ]:
asn_files = sorted(Path(out_dir).glob("*json"))
len(asn_files)

I find that it can be helpful to take the all the PRI headers and put them into a pandas DataFrame. The columns are the header keywords, and each row contains the keyword values for one exposure. This makes it easy to filter the dataset by keyword and figure out which files you want for different steps.

In [ ]:
def organize_files_by_hdr(files, extnum=0):
    """
    Put all the header keywords into a dataframe for sorting and filtering.
    Give it a list of the files, and the extension to take the headers from
    """
    hdrs = []
    for f in data_files:
        hdr = fits.getheader(str(f), 0)
        hdr = pd.Series(hdr)
        # drop duplicated index entries, usually this is "''"" and "COMMENT"
        drop_index = hdr.index[hdr.index.duplicated()]
        hdr.drop(index=drop_index, inplace=True)
        # also explcitly drop all instances of "''" and 'COMMENT'
        for label in ['','COMMENT']:
            try:
                hdr.drop(labels=label)
            except KeyError:
                # probably this means there are no entries with this index label
                pass
        hdrs.append(hdr)
    hdr_df = pd.concat(hdrs, axis=1).T
    return hdr_df

In [ ]:
hdr_df = organize_files_by_hdr(data_files, 0)

In [ ]:
hdr_df.head()

### Prepare the exposures for PSF subtraction

The MIRI coronagraphic observations of HIP-65426 are structured as follows (see APT file):

- 1140 Coronagraph
    - Obs 4: Science target, roll 2
        - 2 target acquisition exposures
        - 1 occulted exposure
    - Obs 5: Science target, roll 1
        - 2 target acquisition exposures
        - 1 occulted exposure
    - Obs 6: Reference target
        - 2 target acquisition exposures
        - 9 occulted exposure (dithered)
    - Obs 28: Science target background exposure
        - 1 background exposure
    - Obs 29: Science target background exposure
        - 1 background exposure
- 1550 Coronagraph
    - Obs 7: Science target, roll 1
        - 2 target acquisition exposures
        - 1 occulted exposure
    - Obs 8: Science target, roll 2
        - 2 target acquisition exposures
        - 1 occulted exposure
    - Obs 9: Reference target
        - 2 target acquisition exposures
        - 9 occulted exposure (dithered)
    - Obs 30: Science target background exposure
        - 1 background exposure
    - Obs 31: Science target background exposure
        - 1 background exposure

Note: Later coronagraphic programs used a 2-point dither pattern for the background, but this was not yet in place when 01386 executed. These programs will have 2 background exposures for each background observation, not 1. 

Now that we have the files, let's split them up for their different roles in PSF subtraction. Header keywords can be helpful, as well as the APT file.

In this example, we're going to use the observation numbers from the APT file to match exposures, but here are some helpful keywords anyway:

- `BGKDTARG` -> True if a background observation, else False
- `EXP_TYPE` -> MIR_TACQ for TA images, otherwise MIR_4QPM and MIR_LYOT for occulted exposures
- `IS_PSF`   -> True if the star is a PSF reference target, False if it's a science target

## 1140 data

First, let's take a quick look at all the 1140 exposures. 

In [ ]:
hdrs_1140 = hdr_df.query("SUBARRAY == 'MASK1140'")
nfiles = len(hdrs_1140)
ncols = int(np.ceil(np.sqrt(nfiles)))
nrows = int(np.ceil(nfiles/ncols))
fig, axes = plt.subplots(figsize=(12,12), ncols=ncols, nrows=nrows)
for irow, ax in zip(hdrs_1140.iterrows(), axes.ravel()):
    i, row = irow
    img = fits.getdata(Path(out_dir) / row['FILENAME'], 1)
    # collapse cubes to 2-D for plotting
    while np.ndim(img) > 2:
        img = img.mean(axis=0)
    norm = np.nanquantile(img, [0.05, 0.95])
    # we will plot images with the convention that the middle of the lower left pixel is (1, 1)
    y, x = [np.linspace(0.5, i+0.5, i+1) for i in img.shape]
    ax.pcolor(x, y, img, vmin=norm[0], vmax=norm[1])
    ax.set_aspect('equal')
    ax.set_title(f"Obs {row['OBSERVTN']} Dith {row['PATT_NUM']}")

# turn off extra plots
for ax in axes.flat[nfiles:]:
    ax.set_visible(False)

In [ ]:
# Get the rows for the science target. We include the EXP_TYPE keyword to filter out the TA exposurs
sci_targ_files = hdr_df.query(f"OBSERVTN in {['004', '005']} and EXP_TYPE == 'MIR_4QPM'") 
# Background observations don't perform TA
sci_bgnd_file = hdr_df.query("OBSERVTN == '028'")

In [ ]:
ref_bgnd_file = hdr_df.query("OBSERVTN == '029'")
ref_targ_files = hdr_df.query(f"OBSERVTN == '006' and EXP_TYPE == 'MIR_4QPM'")

It looks like the backgrounds were subtracted correctly by the pipeline off of the science exposures, but not from the reference exposures. This happens sometimes, so we'll do it ourselves.

Let's subtract the background from the reference target exposures by switching `if False` to `if True` (and then set it back again, you only want to run this once).
We'll overwrite the file with the new background-subtracted image.

Once you're finished with this step, you can regenerate the above plots to see the status of the exposures.

In [ ]:
# Science exposyre background subtraction
# We only want to run background subtraction once. 
# After you run it, set this to False. You can set it back later.
if False:
    bgnd_img = fits.getdata((Path(out_dir) / sci_bgnd_file['FILENAME'].iloc[0]), 1)
    for f in sci_targ_files['FILENAME']:
        with fits.open(Path(out_dir) / f) as hdulist:
            #hdulist.writeto(str(Path(out_dir) / f)+"-bkp")
            sci_img = hdulist[1].data
            sci_sub = sci_img - bgnd_img
            hdulist[1].data = sci_sub
            hdulist.writeto(str(Path(out_dir) / f), overwrite=True)

In [ ]:
# Reference target background subtraction
if False:
    bgnd_img = fits.getdata((Path(out_dir) / ref_bgnd_file['FILENAME'].iloc[0]), 1)
    for f in ref_targ_files['FILENAME']:
        with fits.open(Path(out_dir) / f) as hdulist:
            img = hdulist[1].data
            sub = img - bgnd_img
            hdulist[1].data = sub
            hdulist.writeto(str(Path(out_dir) / f), overwrite=True)


Now you have the glowstick-subtracted science and reference files

Now we're ready to run the Stage 3 pipeline. Let's import it, and set the parameters.

In [ ]:
from jwst.pipeline import Coron3Pipeline

In [ ]:
params = Coron3Pipeline().get_pars()

In [ ]:
params['save_results'] = True
params['output_dir'] = './pipeline_output/1140'

Now you have to select the association file. You want to look for the one pertaining to the 1140 data, and if there are multiple, you want the one with the most recent version of the pipeline.

In [ ]:
# we want the ASN file for the 1140 data. 
asn_filename = "./jw01386-c1021_20230717t003100_coron3_00001_asn.json"

Finally, run the pipeline step. Go get something to drink, because this might take a while.

In [ ]:
cor3 = Coron3Pipeline().call(asn_filename, **params)

## 1550 data

We're going to repeat the same steps as with the 1140 data.

First off, take a look at the different data products

In [ ]:
hdrs_1550 = hdr_df.query("SUBARRAY == 'MASK1550'")
nfiles = len(hdrs_1550)
ncols = int(np.ceil(np.sqrt(nfiles)))
nrows = int(np.ceil(nfiles/ncols))
fig, axes = plt.subplots(figsize=(12,12), ncols=ncols, nrows=nrows)
for irow, ax in zip(hdrs_1550.iterrows(), axes.ravel()):
    i, row = irow
    img = fits.getdata(Path(out_dir) / row['FILENAME'], 1)
    # collapse cubes to 2-D for plotting
    while np.ndim(img) > 2:
        img = img.mean(axis=0)
    norm = np.nanquantile(img, [0.05, 0.95])
    y, x = [np.linspace(0.5, i+0.5, i+1) for i in img.shape]
    ax.pcolor(x, y, img, vmin=norm[0], vmax=norm[1])
    ax.set_aspect('equal')
    ax.set_title(f"Obs {row['OBSERVTN']} Dith {row['PATT_NUM']}")

# turn off extra plots
for ax in axes.flat[nfiles:]:
    ax.set_visible(False)

This time, it looks like only Obs 9 had the background subtracted automatically by the pipeline. We need to remove the glowsticks for Obs 8, as well as for Obs 7 (the reference star).

In [ ]:
# Get the rows for the science target. We include the EXP_TYPE keyword to filter out the TA exposures
sci_targ_files = hdr_df.query(f"OBSERVTN in {['008', '009']} and EXP_TYPE == 'MIR_4QPM'") 
# Background observations don't perform TA
sci_bgnd_file = hdr_df.query("OBSERVTN == '030'")

In [ ]:
ref_targ_files = hdr_df.query(f"OBSERVTN == '007' and EXP_TYPE == 'MIR_4QPM'")
ref_bgnd_file = hdr_df.query("OBSERVTN == '031'")

In [ ]:
# We only want to run background subtraction once. 
# After you run it, set this to False. You can set it back later.
if False:
    bgnd_img = fits.getdata((Path(out_dir) / sci_bgnd_file['FILENAME'].iloc[0]), 1)
    for f in sci_targ_files[sci_targ_files['OBSERVTN'] == '008']['FILENAME']:
        with fits.open(Path(out_dir) / f) as hdulist:
            sci_img = hdulist[1].data
            sci_sub = sci_img - bgnd_img
            hdulist[1].data = sci_sub
            hdulist.writeto(str(Path(out_dir) / f), overwrite=True)

In [ ]:
if False:
    bgnd_img = fits.getdata((Path(out_dir) / ref_bgnd_file['FILENAME'].iloc[0]), 1)
    for f in ref_targ_files['FILENAME']:
        with fits.open(Path(out_dir) / f) as hdulist:
            img = hdulist[1].data
            sub = img - bgnd_img
            hdulist[1].data = sub
            hdulist.writeto(str(Path(out_dir) / f), overwrite=True)


Now that you have the glowstick-subtracted science and reference files, they are ready for PSF subtraction.

In [ ]:
params = Coron3Pipeline().get_pars()

In [ ]:
params['save_results'] = True
params['output_dir'] = './pipeline_output/1550/'

Now you have to select the association file. You want to look for the one pertaining to the 1140 data, and if there are multiple, you want the one with the most recent version of the pipeline.

In [ ]:
# Make sure that the association file you choose lists the correct files, with the correct descriptions
asn_filename = "./jw01386-c1022_20230717t003100_coron3_00001_asn.json"

In [ ]:
cor3 = Coron3Pipeline().call(asn_filename, **params)

## Part 3: Examining the output

`calwebb_coron3` has several data products besides `i2d` and `crfints` files that are common to other pipeline Stage 3 classes. The descriptions are taken from https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html#calwebb-coron3.
- `_crfints.fits`: In these files, the DQ array has been updated to flag pixels detected as outliers.
- `_median.fits`: This is used by the cosmic ray flagging step.
- `_i2d.fits`: The resample step the PSF-subtracted products, usually separated by a roll angle, into a single 2-D image.
- `_psfstack.fits`: The data from each input PSF reference exposure are concatenated into a single combined 3D stack by the `stack_refs` step
- `_psfalign.fits`: For each science target exposure, all of the reference PSF images in the `_psfstack` product are aligned to each science target integration and saved to a 4D `_psfalign` product by the `align_refs` step
- `_psfsub.fits`: For each science target exposure, the klip step applies PSF fitting and subtraction for each integration, resulting in a 3D stack of PSF-subtracted images.

In [ ]:
output_paths = {'1140': "./pipeline_output/1140/",
                '1550': "./pipeline_output/1550/"}

Let's split the files up into the different types

In [ ]:
cosmic_files = {coro: sorted(Path(path).glob("*crfints*")) for coro, path in output_paths.items()}
median_files = {coro: sorted(Path(path).glob("*median*")) for coro, path in output_paths.items()}
resamp_files = {coro: sorted(Path(path).glob("*i2d*")) for coro, path in output_paths.items()}
psfstack_files = {coro: sorted(Path(path).glob("*psfstack*")) for coro, path in output_paths.items()}
psfalign_files = {coro: sorted(Path(path).glob("*psfalign*")) for coro, path in output_paths.items()}
psfsub_files = {coro: sorted(Path(path).glob("*psfsub*")) for coro, path in output_paths.items()}

Here are the final derotated-and-combined PSF subtracted images:

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(16, 8))

ax = axes[0]
ax.set_title("1140")
img = fits.getdata(resamp_files["1140"][0], 1)
while np.ndim(img) > 2:
    img = img.mean(axis=0)
norm = np.nanquantile(img, [0.05, 0.95])
y, x = [np.linspace(0.5, i+0.5, i+1) for i in img.shape]
ax.pcolor(x, y, img, vmin=norm[0], vmax=norm[1])

ax = axes[1]
ax.set_title("1550")
img = fits.getdata(resamp_files["1550"][0], 1)
while np.ndim(img) > 2:
    img = img.mean(axis=0)
norm = np.nanquantile(img, [0.05, 0.95])
y, x = [np.linspace(0.5, i+0.5, i+1) for i in img.shape]
ax.pcolor(x, y, img, vmin=norm[0], vmax=norm[1])


for ax in axes:
    ax.set_aspect("equal")

The remaining images are data cubes and best viewed in a FITS viewer like DS9 or ginga.